In [13]:
import pandas as pd
import statsmodels.api as sm
import matplotlib as plt
df=pd.read_csv('/datc/opschaler/combined_gas_smart_weather_dfs/processed/P01S02W4953_hour.csv',header=0,delimiter='\t')
dwelling_id = 'P01S02W4953'
df.isnull().sum()
df=df.dropna()
df = df.set_index(pd.DatetimeIndex(df['datetime']))
df.head()
 

,datetime,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,gasMeter,gasPower,DD,...,P,Q,RG,SQ,T,T10,TD,U,VV,WW
datetime,,,,,,,,,,,,,,,,,,,,,
2017-07-05 11:00:00,2017-07-05 11:00:00,1470.556030,1389.960327,1475.128052,599.853027,0.000000,1604.250000,1238.140991,0.060059,294.700002,...,1018.250000,516.166667,0.0,6.037200,22.816667,24.983333,15.716667,63.833333,34483.333333,2.833333
2017-07-05 12:00:00,2017-07-05 12:00:00,1470.556152,1391.007935,1475.128052,599.853027,14.714286,596.379700,1238.201050,0.060059,300.549998,...,1018.233337,662.666667,0.0,6.960717,23.466667,25.400000,15.633334,61.166667,33350.000000,2.000000
2017-07-05 13:00:00,2017-07-05 13:00:00,1470.579590,1391.560547,1475.128052,599.853027,11.562675,801.231201,1238.204956,0.003906,285.750000,...,1018.216675,694.500000,0.0,5.372850,23.983333,26.283333,15.466667,58.500000,36666.666667,1.666667
2017-07-05 14:00:00,2017-07-05 14:00:00,1470.579956,1392.567749,1475.128052,599.853027,0.089636,960.882324,1238.204956,0.000000,294.550003,...,1018.250000,812.666667,0.0,9.047567,24.233333,27.116667,15.483333,57.833333,35400.000000,1.666667
2017-07-05 15:00:00,2017-07-05 15:00:00,1470.581055,1393.423828,1475.128052,599.853027,0.000000,837.653625,1238.208984,0.004028,311.250000,...,1018.366669,811.000000,0.0,9.430167,23.950000,26.583334,14.983333,56.833333,35000.000000,1.500000


In [9]:
def create_corr_matrix(df, dwelling_id, annot):
    """
    Pearson correlation coefficient matrix. 
    The Pearson correlation coefficient is a measure of the linear correlation between two variables.
    """
    plt.clf()
    %matplotlib inline
    corr = df.corr()
    mask = np.zeros_like(df.corr())
    mask[np.triu_indices_from(mask)] = True

    if annot:
        fig, ax = plt.subplots(figsize=(25,25))
    else:
        fig, ax = plt.subplots()

    fig = sns.heatmap(corr, mask=mask, square=False, cmap='RdYlGn', annot=annot, ax=ax, 
                cbar_kws={'label':'Pearson correlation coefficient [-]'})

    fig.set_title('Correlation matrix of dwelling ID: '+dwelling_id)
    fig.tick_params(axis='x', rotation=90)
    fig.tick_params(axis='y', rotation=0)

    fig = fig.get_figure()
    fig.tight_layout()
    
    fig.show()
    
    return fig

In [11]:
fig = create_corr_matrix(df, dwelling_id, True)

NameError: name 'plt' is not defined

In [8]:
X = df[['T','Q','eMeter','N']]
Y = df['gasPower']

X = sm.add_constant(X) # adding a constant


model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:               gasPower   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     287.8
Date:                Fri, 05 Oct 2018   Prob (F-statistic):          5.88e-212
Time:                        13:28:19   Log-Likelihood:                 2963.8
No. Observations:                3242   AIC:                            -5918.
Df Residuals:                    3237   BIC:                            -5887.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2767      0.038     -7.275      0.0